In [2]:
from IPython.core.display import display, HTML
import pandas as pd
import numpy as np
import os
import re
import json
import psycopg2
from datetime import date
import presto  # import prestosql python client
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning
warnings.simplefilter('ignore', InsecureRequestWarning)
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.max_columns = None

In [3]:
## connect to cluster
uid = "_svc_acs_presto"
pwd = "em6qF7Gp8E9ECYcKx3gHZ#v$H@pzT7AX3uCJp%5wesKE%VD6gZaCB8*rZ2zNgMKdPkuetSJph2H&H2eadbfCkeB3tBbUqcM^Sj5k"

conn = presto.dbapi.connect(
    host='acs-exports.bi.ghdna.io',
    port=8446,
    user='_svc_acs_presto',
    catalog='hive',
    schema='default',
    http_scheme='https',
    verify=False,
    auth=presto.auth.BasicAuthentication(uid, pwd)
)
cur = conn.cursor()

## Query Flowcell QC and sequencer information for each flowcell 

In [ ]:
vw_batch_query = '''
select 
acs_instance,
connection_id,
inventory_id as flowcell_id,
name as batch_name
from vw_batch 
'''

vw_batch = []
for r in cur.execute(vw_batch_query):
     vw_batch.append(r)
        
colnames = [desc[0] for desc in cur.description]
vw_batch_df = pd.DataFrame(vw_batch, columns=colnames)


flowcell_query = '''
select 
acs_instance,
flowcell_id,
product,
sequencer_name,
created
from vw_flowcell 
ORDER BY created DESC
'''

vw_flowcell = []
for r in cur.execute(flowcell_query):
     vw_flowcell.append(r)
        
colnames = [desc[0] for desc in cur.description]
flowcell_df = pd.DataFrame(vw_flowcell, columns=colnames)

flowcell_df['created'] = pd.to_datetime(flowcell_df['created'])
flowcell_df['pacific_time'] = flowcell_df['created'].dt.tz_convert('US/Pacific')


def get_conn(db): 
    conn = psycopg2.connect(db)
    return conn

def get_cursor(conn):
    cursor = conn.cursor()
    return cursor

def statement_to_df(cursor):
    df = pd.DataFrame(cursor.fetchall(),columns = [i[0] for i in cursor.description])
    return df

prod = "host='10.4.170.26' dbname='ghdb' user='admin' password='N7Tks0xPS'" 


GHDB_prod_conn = get_conn(prod)
GHDB_prod_cursor = get_cursor(GHDB_prod_conn)

query = '''
SELECT DISTINCT
*
FROM qc_seq
'''

GHDB_prod_cursor.execute(query, )
df = statement_to_df(GHDB_prod_cursor)


df['flowcell_id'] = df['runid'].str.split('_').str[3].str.slice(1,10)

flowcell_df_merge = flowcell_df.merge(df, on = ['flowcell_id'], how = 'inner')
flowcell_df_merge = flowcell_df_merge[flowcell_df_merge['sequencer_name'].notnull()]
flowcell_df_merge = flowcell_df_merge.merge(vw_batch_df, on = ['flowcell_id', 'acs_instance'], how = 'inner')


## Save the data into csv file on HPC
todaydate = date.today()
todaydate_ref = todaydate.strftime("%Y%m%d")

file_path = '/ghds/groups/labdesk/jiwang/analysis/acs_exports/'
file_name = '_omni_g360_flowcells.csv'
flowcell_df_merge.to_csv(file_path + todaydate_ref + file_name, index=False)